EPL: league 39

In [3]:
import requests
import json
import os

from dotenv import load_dotenv
load_dotenv()


ModuleNotFoundError: No module named 'requests'

# Get Scores from EPL on date

In [56]:
# get all the ids of a match for a url 
url = "https://v3.football.api-sports.io/fixtures"

payload={'league':39, 'season':2022, 'date': '2022-08-06'}

headers = {
    'x-rapidapi-key': os.environ.get("RAPIDAPI_KEY"),
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

fixtures_response = json.loads(requests.request("GET", url, headers=headers, params=payload).text)


In [174]:
fixtures_response

{'get': 'fixtures',
 'parameters': {'league': '39', 'season': '2022', 'date': '2022-08-06'},
 'errors': [],
 'results': 6,
 'paging': {'current': 1, 'total': 1},
 'response': [{'fixture': {'id': 867947,
    'referee': 'A. Madley',
    'timezone': 'UTC',
    'date': '2022-08-06T11:30:00+00:00',
    'timestamp': 1659785400,
    'periods': {'first': 1659785400, 'second': 1659789000},
    'venue': {'id': 535, 'name': 'Craven Cottage', 'city': 'London'},
    'status': {'long': 'Match Finished', 'short': 'FT', 'elapsed': 90}},
   'league': {'id': 39,
    'name': 'Premier League',
    'country': 'England',
    'logo': 'https://media.api-sports.io/football/leagues/39.png',
    'flag': 'https://media.api-sports.io/flags/gb.svg',
    'season': 2022,
    'round': 'Regular Season - 1'},
   'teams': {'home': {'id': 36,
     'name': 'Fulham',
     'logo': 'https://media.api-sports.io/football/teams/36.png',
     'winner': None},
    'away': {'id': 40,
     'name': 'Liverpool',
     'logo': 'https://

In [153]:
score_match_dict = {}
for match in fixtures_response['response']:
    # get the fixture id
    fixture = match['fixture']['id']
    
     # get the team names
    home_team = match['teams']['home']['name']
    away_team = match['teams']['away']['name']

    # get the score
    home_score = match['score']['fulltime']['home']
    away_score = match['score']['fulltime']['away']

    # who won?
    if home_score > away_score:
        winner = 'Home'
    elif home_score < away_score:
        winner = 'Away'
    else:
        winner = 'Draw'

    score_match_dict[fixture]={'home_team':home_team, 'away_team':away_team, 'home_score':home_score, 'away_score':away_score, 'winner':winner}


Fulham: 2 - Liverpool: 2
Bournemouth: 2 - Aston Villa: 0
Leeds: 2 - Wolves: 1
Newcastle: 2 - Nottingham Forest: 0
Tottenham: 4 - Southampton: 1
Everton: 0 - Chelsea: 1


# Get the "Odds of Winning" for each game on the date

In [154]:

url = "https://v3.football.api-sports.io/odds"

payload={'league':39, 'season':2022, 'date': '2022-08-06'}

headers = {
    'x-rapidapi-key': "1e5964a18e1b9f63d1deeda23247e29e",
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

odds_response = json.loads(requests.request("GET", url, headers=headers, params=payload).text)


In [155]:
odds_match_dict = {}
for match in odds_response['response']:

    # check to make sure 0 == match winner
    if match['bookmakers'][0]['bets'][0]['name'] != 'Match Winner':
        print('oh no! first id is not match winner')

    odd_dict = {}
    for value in match['bookmakers'][0]['bets'][0]['values']:
        odd_dict[value['odd']] = value['value']

    pred_win = odd_dict[max(odd_dict.keys())]

    
    
    odds_match_dict[match['fixture']['id']] = pred_win

In [176]:
odds_match_dict

{867947: 'Draw',
 867948: 'Home',
 867949: 'Draw',
 867951: 'Away',
 867952: 'Away',
 867953: 'Home'}

# Print Outputs

In [172]:
# get lists of correct and incorrect predictions
correct_ids = []
upset_ids = []
for match_id in odds_match_dict.keys():
    if odds_match_dict[match_id]!=score_match_dict[match_id]['winner']:
       upset_ids.append(match_id) 
    else:
        correct_ids.append(match_id)

print("UPSETS:")
for id in upset_ids:
    score_match_dict[id]
    print(str("{home_team}: {home_score} - {away_team}: {away_score}").format(
        home_team=score_match_dict[id]['home_team'], 
        home_score=score_match_dict[id]['home_score'], 
        away_team=score_match_dict[id]['away_team'], 
        away_score=score_match_dict[id]['away_score']))

print("\nEXPECTED:")
for id in correct_ids:
    score_match_dict[id]
    print(str("{home_team}: {home_score} - {away_team}: {away_score}").format(
        home_team=score_match_dict[id]['home_team'], 
        home_score=score_match_dict[id]['home_score'], 
        away_team=score_match_dict[id]['away_team'], 
        away_score=score_match_dict[id]['away_score']))

UPSETS:
Leeds: 2 - Wolves: 1
Newcastle: 2 - Nottingham Forest: 0
Tottenham: 4 - Southampton: 1
Everton: 0 - Chelsea: 1

EXPECTED:
Fulham: 2 - Liverpool: 2
Bournemouth: 2 - Aston Villa: 0
